In [ ]:
%load_ext autoreload
%autoreload 2
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
import argparse
import dataclasses
import sys
import timeit
from typing import Tuple
import pickle
import click
import matplotlib.pyplot as plt
import numpy as np
import open3d
import torch
from tqdm import tqdm


# from home_robot.mapping.voxel import SparseVoxelMap
from home_robot.utils.point_cloud_torch import unproject_masked_depth_to_xyz_coordinates

from pytorch3d.vis.plotly_vis import AxisArgs, plot_scene, get_camera_wireframe
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
torch.set_printoptions(sci_mode=False)



In [ ]:
from home_robot.datasets.eqa.dataset import EQADataset
config_dict = dict(
    dataset_name = 'eqa',
    camera_params = dict(
        image_height=1080,
        image_width=1920,
        # png_depth_scale = 1000.0 #for depth image in png format
        png_depth_scale = 6553.5 #for depth image in png format
    )
)

dataset = EQADataset(
        config_dict,
        # '/checkpoint/maksymets/eaif/datasets/eqa-v2/frames/scannet-v0/',
        # '108-scannet-scene0354_00',
        '/checkpoint/maksymets/eaif/datasets/eqa-v2/frames/hm3d-v0/',
        '000-hm3d-BFRyYbPCCPE',
        desired_height=480,
        desired_width=853,
        stride = 1,
        device='cpu'
)

In [ ]:
# short_dataset = [dataset[i] for i in range(3)]
short_dataset = [v for v in dataset]

In [ ]:
rgb, depth, Ks, pose = zip(*short_dataset)
rgb, depth, Ks, pose = [torch.stack(v) for v in [rgb, depth, Ks, pose]]
print(len(dataset))
plt.imshow(rgb[0].cpu() / 255.)
plt.show()
plt.imshow(rgb[1].cpu() / 255.)
plt.show()

In [ ]:
v = torch.tensor([[1., 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
v.inverse() - v

In [ ]:
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_scene, get_camera_wireframe
from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes

unprojected = unproject_masked_depth_to_xyz_coordinates(
    # depth = depth[0, None].unsqueeze(1),
    # pose = poses_opencv[0, None],
    depth = depth.unsqueeze(1).squeeze(-1),
    pose = pose,
    inv_intrinsics = torch.linalg.inv(Ks)[:, :3, :3],
    # mask: Optional[torch.Tensor] = None,
) 

ptc = Pointclouds(
    [unprojected.reshape(-1, 3)],
    features = [rgb.reshape(-1,3) / 255.],
).subsample(100000)

fig = plot_scene({
    "global scene": dict(
        ptc=ptc
    )
    },
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=200_000,
    height=1000,
    # width=1000,
)

add_camera_poses(fig, pose)
fig.update_layout(
    # width=width,
    height=1000,
    # aspectmode="data"
)

In [ ]:
# dataset.rawpose[1]
pose[1]